<center>
<p style="text-align:center"><img alt="Ragas" src="https://github.com/explodinggradients/ragas/blob/main/docs/_static/imgs/logo.png?raw=true" width="400"><br><a href="https://arize.com/docs/phoenix/">Phoenix Docs</a> | <a href="https://github.com/explodinggradients/ragas">Ragas</a> | <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
</p>
    </center>

## 1. Introduction

Building a baseline for a RAG pipeline is not usually difficult, but enhancing it to make it suitable for real-world Arabic question-answering tasks requires careful evaluation, observability, and iterative refinement. Choosing the right tools, retrieval strategies, and LLM configurations can be challenging, especially when working with large and diverse datasets.

This notebook demonstrates how to **evaluate, visualize, and analyze an Arabic RAG pipeline** using a combination of powerful open-source libraries:

- **[Ragas](https://docs.ragas.io/en/stable/)** for evaluation metrics such as faithfulness, answer correctness, context precision, and context recall  
- **Arize AI’s [Phoenix](https://arize.com/docs/phoenix)** for tracing, debugging, embedding visualization, and cluster analysis  
- **[LlamaIndex](https://docs.llamaindex.ai/en/stable/)** for building RAG pipelines, managing document indexes, and connecting retrieval to LLMs  

In this tutorial, instead of using arXiv PDFs, we will work with the **ArabicaQA dataset**, a large-scale Arabic machine-reading comprehension (MRC) dataset containing questions, contexts, and ground-truth answers. This allows us to build and evaluate an **Arabic-focused RAG system** in a realistic setting.

> ℹ️ **Note:**  
> This notebook uses **open-source HuggingFace LLMs** (such as LLaMA or Gemma) and **does not require an OpenAI API key**.


## 2. Install Dependencies and Import Libraries

Install and import Python dependencies.

In [1]:
%pip install -U "ragas==0.1.4" pypdf "arize-phoenix[llama-index,embeddings]" "openai>=1.0.0" pandas "httpx<0.28" "openinference-instrumentation>=0.1.38"


  Using cached ragas-0.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached pypdf-6.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached openai-2.8.1-py3-none-any.whl.metadata (29 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached openinference_instrumentation-0.1.42-py3-none-any.whl.metadata (5.0 kB)
  Using cached arize_phoenix-12.16.0-py3-none-any.whl.metadata (35 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached langchain-1.0.8-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_openai-1.0.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosqlite-0.21.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached alembic-1.17.2-py3-none-any.whl.metadata (7.2 kB)
  Using cached arize_phoenix_client-1.24.0-py3-none-any.whl.metadata (11 kB)
  Using cached arize_phoenix_evals-2.6.0-py

In [6]:
!pip install "llama-index>=0.11.0" "llama-index-llms-huggingface"

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
    --------------------------------------- 0.3/11.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.9 MB 1.6 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/11.9 MB 1.6 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/11.9 MB 1.7 MB/s eta 0:00:

In [1]:
import torch
print("CUDA:", torch.cuda.is_available())
print("Torch CUDA build:", torch.version.cuda)
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA: True
Torch CUDA build: 12.1
GPU: NVIDIA GeForce GTX 1660 Ti


In [2]:
import pandas as pd

# Display the complete contents of dataframe cells.
pd.set_option("display.max_colwidth", None)

In [3]:
from llama_index.core.settings import Settings
from llama_index.llms.huggingface import HuggingFaceLLM

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Reset any previous LLM in LlamaIndex
Settings.llm = None  

model_name = "meta-llama/Llama-3.2-1B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print("Using device:", device)

# --- Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# --- Model ---
if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=dtype,
        device_map="auto",
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=dtype,
    )

print("First model parameter device:", next(model.parameters()).device)

# --- Wrap in LlamaIndex LLM ---
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    model_name=model_name,
    tokenizer_name=model_name,
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={
        "do_sample": False,   # greedy, deterministic (good for evaluation)
    },
)

from ragas.llms import LlamaIndexLLMWrapper
evaluator_llm = LlamaIndexLLMWrapper(llm)   # deprecation warning is OK for now

# Register with LlamaIndex (if you use it elsewhere)
Settings.llm = llm


C:\Users\AZhaa\anaconda3\envs\nlp_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LLM is explicitly disabled. Using MockLLM.
Using device: cuda


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct.
403 Client Error. (Request ID: Root=1-693b3b40-05fbd94931edb91a484df44e;eaa73c62-bfbf-4db1-8f8e-07e761085e65)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct to ask for access.

In [ ]:
import phoenix as px

(session := px.launch_app()).view()

In [11]:
!pip install openinference-instrumentation-langchain openinference-instrumentation-llama-index


   --------- ------------------ 1/3 [openinference-instrumentation-llama-index]
   ------------------------------ 3/3 [openinference-instrumentation-langchain]



In [33]:
from phoenix.otel import register
from openinference.instrumentation.langchain import LangChainInstrumentor

tracer_provider = register()
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

print("✅ LangChain tracing enabled!")
print("⚠️  LlamaIndex tracing skipped due to version compatibility")

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

✅ LangChain tracing enabled!
⚠️  LlamaIndex tracing skipped due to version compatibility


## 4. Load our dataset (ArabicaQA)

In [34]:
from datasets import load_dataset

# Load the MRC split of ArabicaQA from Hugging Face
arabica_mrc = load_dataset(
    "abdoelsayed/ArabicaQA",
    data_files={
        #"train": "MRC/train.json",
        #"validation": "MRC/val.json",
        "test": "MRC/test.json",
    },
    field="data",  # the JSON has a top-level field called "data"
)

arabica_mrc


DatasetDict({
    test: Dataset({
        features: ['paragraphs'],
        num_rows: 2108
    })
})

In ArabicaQA (MRC task):
The MRC test set contains:
- question
-  context (the correct paragraph from Wikipedia)
-  answers (ground truth answer spans)

We will use:
- context → to build your vector index (RAG knowledge base)
- question → to query your RAG
- answers → ground truth for RAGAS evaluation

In [35]:
test_ds = arabica_mrc["test"]
test_ds[0]


{'paragraphs': [{'context': 'المرصد الحضري لمدينة الرياض ويعرف اختصارًا بمرصد الرياض هو مركز متخصص في وضع نظم مراقبة حضرية لرصد سير عمليات التنمية الحضرية للمدينة، ويهدف إلى فهم الوضع الحالي للمدينة ووضع تصور للاتجاهات المستقبلية، ومراقبة الإنجاز لتحقيق الأهداف واتخاذ القرارات. التأسيس. بدأت أول خطوات إنشاء المرصد في عام 2009م، من الهيئة العليا لتطوير مدينة الرياض؛ ومن ثم أُقرَّ الهيكل التنظيمي للمرصد الحضري الذي يتكون من مجلس ولجنة تنفيذية ومركز. ويترأس مجلس المرصد أمير المنطقة بالإضافة إلى أربعة عشر عضوا، وتتكون اللَّجنة التنفيذية من 15 عضوًا يمثلون أهم مصادر البيانات ذات الصلة بالمؤشرات الحضرية من القطاع الحكومي والقطاع الخاص. المهام. في عام 1434هـ أطلق المرصد الحضري لمدينة الرياض، المؤشرات الحضرية للدورة الأولى التي ضمت (80) مؤشراً، بعد إقرار نتائج تلك المؤشرات في اجتماع الهيئة الثاني لعام 1435هـ، وشملت الحدّ الأدنى من المؤشرات الحضرية العالمية بالإضافة إلى المؤشرات المحلية. يعمل المرصد الحضري لمدينة الرياض على رصد وتقييم اتجاهات التنمية في مدينة الرياض من خلال إنتاج المؤشرات الحضر

## Convert the test set into a clean RAG-ready DataFrame
For RAG, we need:
- knowledge documents → contexts
- evaluation questions → questions
- ground truth → answers

ArabicaQA test samples have 1 context + multiple Q/A pairs, so we must flatten them.

In [36]:
import pandas as pd

def flatten_arabica_mrc(dataset):
    rows = []

    for item in dataset:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answers = qa["answers"]
                # Most have exactly 1 answer
                gt_answer = answers[0]["text"] if len(answers) > 0 else ""

                rows.append({
                    "context": context,
                    "question": question,
                    "ground_truth": gt_answer
                })
    return pd.DataFrame(rows)


df_test = flatten_arabica_mrc(test_ds)
df_test.head()


,context,question,ground_truth
0,المرصد الحضري لمدينة الرياض ويعرف اختصارًا بمرصد الرياض هو مركز متخصص في وضع نظم مراقبة حضرية لرصد سير عمليات التنمية الحضرية للمدينة، ويهدف إلى فهم الوضع الحالي للمدينة ووضع تصور للاتجاهات المستقبلية، ومراقبة الإنجاز لتحقيق الأهداف واتخاذ القرارات. التأسيس. بدأت أول خطوات إنشاء المرصد في عام 2009م، من الهيئة العليا لتطوير مدينة الرياض؛ ومن ثم أُقرَّ الهيكل التنظيمي للمرصد الحضري الذي يتكون من مجلس ولجنة تنفيذية ومركز. ويترأس مجلس المرصد أمير المنطقة بالإضافة إلى أربعة عشر عضوا، وتتكون اللَّجنة التنفيذية من 15 عضوًا يمثلون أهم مصادر البيانات ذات الصلة بالمؤشرات الحضرية من القطاع الحكومي والقطاع الخاص. المهام. في عام 1434هـ أطلق المرصد الحضري لمدينة الرياض، المؤشرات الحضرية للدورة الأولى التي ضمت (80) مؤشراً، بعد إقرار نتائج تلك المؤشرات في اجتماع الهيئة الثاني لعام 1435هـ، وشملت الحدّ الأدنى من المؤشرات الحضرية العالمية بالإضافة إلى المؤشرات المحلية. يعمل المرصد الحضري لمدينة الرياض على رصد وتقييم اتجاهات التنمية في مدينة الرياض من خلال إنتاج المؤشرات الحضرية التي تقدم معلومات موجزة عن الوضع الراهن والاتجاهات المستقبلية المتوقعة وتقيس مستوى الاستجابة للسياسات والبرامج التنفيذية حسب الأهداف لخطط التنمية المستدامة، ويتم ذلك ضمن إطار رصد عالمي يوفر خطة شاملة للرصد والتقييم ويسهل من عملية تحديد المؤشرات الأساسية ضمن مجموعات معيارية تُمكِّن من مقارنة التقدم مع المدن الأخرى ومؤشرات أخرى تعكس خصوصية مدينة الرياض من خلال تحديد الأولويات والقضايا الهامة في المدينة حسب أهداف المخطط الاستراتيجي الشامل لمدينة الرياض، ورؤية المملكة 2030، بالإضافة إلى أولويات الجهات المشاركة بالمرصد التي تشمل القطاع الحكومي والقطاع الخاص والمجتمع المدني. الإنجازات. في مارس 2017 حصل المرصد الحضري بمدينة الرياض على الشهادة الذهبية WCCD ISO 37120 من المجلس العالمي لبيانات المدن في كندا (WCCD).,من الذي يترأس المرصد الحضري لمدينة الرياض؟,يترأس مجلس المرصد أمير المنطقة بالإضافة إلى أربعة عشر عضوا،
1,المرصد الحضري لمدينة الرياض ويعرف اختصارًا بمرصد الرياض هو مركز متخصص في وضع نظم مراقبة حضرية لرصد سير عمليات التنمية الحضرية للمدينة، ويهدف إلى فهم الوضع الحالي للمدينة ووضع تصور للاتجاهات المستقبلية، ومراقبة الإنجاز لتحقيق الأهداف واتخاذ القرارات. التأسيس. بدأت أول خطوات إنشاء المرصد في عام 2009م، من الهيئة العليا لتطوير مدينة الرياض؛ ومن ثم أُقرَّ الهيكل التنظيمي للمرصد الحضري الذي يتكون من مجلس ولجنة تنفيذية ومركز. ويترأس مجلس المرصد أمير المنطقة بالإضافة إلى أربعة عشر عضوا، وتتكون اللَّجنة التنفيذية من 15 عضوًا يمثلون أهم مصادر البيانات ذات الصلة بالمؤشرات الحضرية من القطاع الحكومي والقطاع الخاص. المهام. في عام 1434هـ أطلق المرصد الحضري لمدينة الرياض، المؤشرات الحضرية للدورة الأولى التي ضمت (80) مؤشراً، بعد إقرار نتائج تلك المؤشرات في اجتماع الهيئة الثاني لعام 1435هـ، وشملت الحدّ الأدنى من المؤشرات الحضرية العالمية بالإضافة إلى المؤشرات المحلية. يعمل المرصد الحضري لمدينة الرياض على رصد وتقييم اتجاهات التنمية في مدينة الرياض من خلال إنتاج المؤشرات الحضرية التي تقدم معلومات موجزة عن الوضع الراهن والاتجاهات المستقبلية المتوقعة وتقيس مستوى الاستجابة للسياسات والبرامج التنفيذية حسب الأهداف لخطط التنمية المستدامة، ويتم ذلك ضمن إطار رصد عالمي يوفر خطة شاملة للرصد والتقييم ويسهل من عملية تحديد المؤشرات الأساسية ضمن مجموعات معيارية تُمكِّن من مقارنة التقدم مع المدن الأخرى ومؤشرات أخرى تعكس خصوصية مدينة الرياض من خلال تحديد الأولويات والقضايا الهامة في المدينة حسب أهداف المخطط الاستراتيجي الشامل لمدينة الرياض، ورؤية المملكة 2030، بالإضافة إلى أولويات الجهات المشاركة بالمرصد التي تشمل القطاع الحكومي والقطاع الخاص والمجتمع المدني. الإنجازات. في مارس 2017 حصل المرصد الحضري بمدينة الرياض على الشهادة الذهبية WCCD ISO 37120 من المجلس العالمي لبيانات المدن في كندا (WCCD).,ما هي الإنجازات البارزة التي حققها المرصد الحضري بمدينة الرياض؟,في مارس 2017 حصل المرصد الحضري بمدينة الرياض على الشهادة الذهبية WCCD ISO 37120 من المجلس العالمي لبيانات المدن في كندا (WCCD)
2,المرصد الحضري لمدينة الرياض ويعرف اختصارًا بمرصد الرياض هو مركز متخصص في وضع نظم مراقبة حضرية لرصد سير عمليات التنمية الحضرية للمدينة، ويهدف إلى فهم الوضع الحالي للمدينة ووضع تصور للاتجاهات المستقبلية، ومراقبة الإنجاز لتحقيق الأهداف واتخاذ القرارات. التأسيس. بدأت أول خطوات إنشاء المرصد في ع

In [37]:
df_test.shape

(13970, 3)

In [38]:
import pandas as pd

# Shuffle the dataset safely
df_shuffled = df_test.sample(frac=1, random_state=42)

# Take a representative sample
df_sample = df_shuffled.head(1000)

df_sample.shape


(1000, 3)

## Deduplicate contexts → build RAG documents

In [39]:
# Make a list of unique contexts (documents)
unique_contexts = df_sample["context"].drop_duplicates().tolist()

len(unique_contexts)


758

In [40]:
print(unique_contexts[0])
print("---------------------------------------")
print(unique_contexts[1])
print("---------------------------------------")
print(unique_contexts[2])


السيخية (بالبنجابيَّة: ਸਿੱਖੀ) هي ديانة توحيدية دارميَّة نشأت في شمالي الهند في نهاية القرن الخامس عشر. وتأتي كلمة «سيخية» من كلمة «سيخ» وهي بدورها تأتي من الجذر السنسكريتي التي تعني التلميذ و في اللغة البالية المريد أو التابع. وهي واحدة من أحدث الأديان الرئيسية في العالم، وهي واحدة من أكبر الديانات في العالم. وتتضمن المعتقدات الأساسية للسيخية، والتي وضِحَت في كتابهم المقدس جورو جرانث صاحب، الإيمان والتأمل في اسم الخالق الواحد، والوحدة الإلهية والمساواة للبشرية جمعاء، والانخراط في خدمة نكران الذات، والسعي لتحقيق العدالة لمصلحة الجميع وازدهارها، وإتباع سلوك معيشة صادق. وفي أوائل القرن الواحد والعشرين كان هناك حوالي 25 مليون سيخي في جميع أنحاء العالم، وتعيش الغالبية العظمى أو 76% (20 مليون) من السيخ في البنجاب، موطن السيخ في شمال غرب الهند، ويعيش حوالي مليوني في الدول الهندية المجاورة، والتي كانت جزءاً من ولاية البنجاب الهندية سابقاً. وسبب انتشارها في العالم هو اعتماد الإنجليز عليهم في بعض الحروب وهجرات السيخ خارج بلادهم، حيث بدأت الهجرة السيخية من الهند البريطانية خلال النصف الثاني من ال

## Build the Vector Index (Embedding + Index + QueryEngine)


In [15]:
import sys
print(sys.executable)

%pip show llama-index


C:\Users\AZhaa\anaconda3\envs\nlp_gpu\python.exe
Name: llama-index
Version: 0.14.8
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License-Expression: MIT
Location: c:\users\azhaa\anaconda3\envs\nlp_gpu\lib\site-packages
Requires: llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install "llama-index-embeddings-huggingface>=0.1.4"


   ---------------------------------------- 0/2 [sentence-transformers]
   ---------------------------------------- 0/2 [sentence-transformers]
   ---------------------------------------- 0/2 [sentence-transformers]
   ---------------------------------------- 0/2 [sentence-transformers]
   ---------------------------------------- 0/2 [sentence-transformers]
   ---------------------------------------- 0/2 [sentence-transformers]
   ---------------------------------------- 0/2 [sentence-transformers]
   ---------------------------------------- 0/2 [sentence-transformers]
   ----------------------------------- 2/2 [llama-index-embeddings-huggingface]



In [41]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    device="cuda" if torch.cuda.is_available() else "cpu",
)

Settings.embed_model = embed_model

test_vec = embed_model.get_text_embedding("hello world")
print("Test embedding length:", len(test_vec))


Test embedding length: 384


## Create LlamaIndex Documents from ArabicaQA contexts

In [42]:
from llama_index.core import Document

documents = [Document(text=c) for c in unique_contexts]
len(documents)


758

## Build the RAG Vector Store using HuggingFace Embeddings

In [43]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.core import Document

def build_query_engine(documents):

    # reset old config
    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.callback_manager = None

    # create the index
    vector_index = VectorStoreIndex.from_documents(
        documents,
        embed_model=embed_model
    )

    # query engine
    query_engine = vector_index.as_query_engine(similarity_top_k=2)

    return query_engine

query_engine = build_query_engine(documents)



## Generate model answers for the test questions

In [44]:
model_answers = []

for q in df_sample["question"]:
    try:
        response = query_engine.query(q)
        model_answers.append(response.response)
    except Exception as e:
        print("Error:", e)
        model_answers.append("")

# Add model answers to dataframe
df_sample["model_answer"] = model_answers

df_sample.head()


KeyboardInterrupt: 

In [47]:
import pandas as pd

completed_n = len(model_answers)  # should be 328
df_sample = df_sample.reset_index(drop=True)

# Attach the answers ONLY to the first completed_n rows
df_sample.loc[:completed_n-1, "model_answer"] = model_answers

# Keep only rows that actually have a model answer (not NaN / empty)
df_done = df_sample.iloc[:completed_n].copy()
df_done = df_done[df_done["model_answer"] != ""]

print(df_done.shape)
df_done.head()


(328, 4)


context  \
0  السيخية (بالبنجابيَّة: ਸਿੱਖੀ) هي ديانة توحيدية دارميَّة نشأت في شمالي الهند في نهاية القرن الخامس عشر. وتأتي كلمة «سيخية» من كلمة «سيخ» وهي بدورها تأتي من الجذر السنسكريتي التي تعني التلميذ و في اللغة البالية المريد أو التابع. وهي واحدة من أحدث الأديان الرئيسية في العالم، وهي واحدة من أكبر الديانات في العالم. وتتضمن المعتقدات الأساسية للسيخية، والتي وضِحَت في كتابهم المقدس جورو جرانث صاحب، الإيمان والتأمل في اسم الخالق الواحد، والوحدة الإلهية والمساواة للبشرية جمعاء، والانخراط في خدمة نكران الذات، والسعي لتحقيق العدالة لمصلحة الجميع وازدهارها، وإتباع سلوك معيشة صادق. وفي أوائل القرن الواحد والعشرين كان هناك حوالي 25 مليون سيخي في جميع أنحاء العالم، وتعيش الغالبية العظمى أو 76% (20 مليون) من السيخ في البنجاب، موطن السيخ في شمال غرب الهند، ويعيش حوالي مليوني في الدول الهندية المجاورة، والتي كانت جزءاً من ولاية البنجاب الهندية سابقاً. وسبب انتشارها في العالم هو اعتماد الإنجليز عليهم في بعض الحروب وهجرات السيخ خارج بلادهم، حيث بدأت الهجرة السيخية من الهند البريطانية خلال النصف الثاني من القرن التاسع عشر، عندما أكمل البريطانيون ضمهم للبنجاب.\nتستند السيخية على التعاليم الروحية لمؤسس الديانة وهو الغورو ناناك، وخلفائه التسعة من الغورو البشر. لقب غورو يعني بالهندية المعلم. أما الغورو غورو جوبيند سينغ الملقب بالعاشر، ساهم في الكثير من أجل السيخية، وكان إسهامه في الإضفاء المستمر للطابع الرسمي على الديانة التي أسسها أولًا الغورو السيخ ناناك ديف جي في القرن الخامس عشر إسهامًا جديرًا بالملاحظة. وسمى الكتاب المقدس للسيخ جورو جرانث صاحب كخليفة له، وبالتالي أنهى خط الغورو البشر وجعل ن الكتاب المقدس للسيخ جورو جرانث صاحب الدليل الروحي الديني والدينوي للسيخ. وترفض الديانة السيخية الادعاءات بأن أي تقليد ديني معين له احتكار للحقيقة المطلقة. وتطورت السيخية في أوقات الاضطهاد الديني. حيث تعرض اثنان من أتباع السيخ وهم الغورو أرجان والغورو تيج بهادور للتعذيب وأعدم من قبل حكام المغول بعد رفضهم اعتناق الإسلام. وأثار اضطهاد السيخ تأسيس الخالسا كطلب لحماية حرية الضمير والدين.\nالتاريخ.\nيرتبط تاريخ السيخية ارتباطًا وثيقًا بتاريخ منطقة البنجاب والوضع الاجتماعي السياسي في شمال غرب شبه القارة الهندية في القرن السادس عشر. منذ الحكم المغولي للهند على يد الإمبراطور جهانكير (1605-1707)، كانت السيخية في صراع مع قوانين إمبراطورية المغول، لأنها كانت تؤثر في التعاقب السياسي للمغول في حين تعتز بالأولياء من الإسلام. قُتل العديد من السيخ البارزين على يد حكام المغول لرفضهم الانصياع لأوامرهم، ومعارضتهم لاضطهاد السيخ. من مجموع 10 من الغورو السيخ، عُذب وأُعدم اثنان من المعلمين أنفسهم (الغورو أرجان والغورو تيج بهادور)، وأقرباء مقربين للعديد من الغورو السيخ قُتلو بوحشية دون رحمة (مثل أبناء الغورو جوبيند سينغ البالغين من العمر 6 و9 سنوات)، إلى جانب العديد من الشخصيات الرئيسية الأخرى السيخية التي عُذبت وقُتلت (مثل باندا بهادور، بهاي ماتي داس، بهاي ساتي داس وباهاي ديالا)، على يد الحكام المغول المتجبرين لرفضهم الخضوع لأوامرهم، ومعارضتهم لاضطهاد السيخ والهندوس. بعد ذلك، عسكرت السيخية لمعارضة هيمنة المغول على أرضهم.\nتميّز ظهور الكونفدرالية السيخية تحت حكم الأمراء والسيخ تحت حكم المهراجا رانجيت سينغ بالتسامح الديني والتعايش السلمي والتعددية مع المسيحيين والمسلمين والهندوس في مواقع السلطة. يعد تأسيس إمبراطورية السيخ عادةً ذروة السيخية على المستوى السياسي، خلال هذه الفترة جاءت إمبراطورية السيخ لتشمل كشمير (أقصى شمال شبه القارة الهندية)، ولداخ (إقليم اتحادي تديره الهند)، وبيشاور (هي عاصمة مقاطعة خيبر بختونخوا الباكستانية وأكبر مدنها). اعتنق عدد من الفلاحين المسلمين والهندوس السيخية. أخذ هاري سينغ نالوا، القائد العام لجيش السيخ على طول الحدود الشمالية الغربية، حدود إمبراطورية السيخ إلى مصب ممر خيبر (هو ممر جبلي في شمال غرب باكستان، على الحدود مع أفغانستان). دمجت الإدارة العلمانية لإمبراطورية السيخ إصلاحات عسكرية واقتصادية وحكومية مبتكرة.\nشهدت الأشهر التي سبقت تقسيم الهند (تقسيم الهند البريطانية إلى دولتين مستقلتين، الهند وباكستان) سنة 1947، صراعًا حادًا في البنجاب (منطقة جيوسياسية وثقافية وتاريخية في جنوب آسيا، تحديدًا شمال شبه القارة الهندية، وتضم مناطق في شرق باكستان وشمال الهند) بين السيخ والمسلمين، شهد الهجرة الدينية الفعالة للسيخ البنجاب والهندوس من البنجاب الغربية مقابل هجرة دينية مماثلة للمسلمين البنجاب في شرق البنجاب. في الوقت الحاضر، يعيش غالبية السيخ في ولاية 

In [48]:
df_done.to_csv("df_sample_with_answers_328.csv", index=False)


In [18]:
print(session.url)


http://localhost:6006/


## RAGAS Evaluation

In [27]:
!pip install -U "ragas[langchain]" langchain-core langchain pydantic



   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.6 MB 3.4 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/9.6 MB 3.2 MB/s eta 0:00:03
   --------- ------------------------------ 2.4/9.6 MB 3.4 MB/s eta 0:00:03
   ------------- -------------------------- 3.1/9.6 MB 3.2 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.6 MB 3.4 MB/s eta 0:00:02
   -------------------- ------------------- 5.0/9.6 MB 3.5 MB/s eta 0:00:02
   ------------------------- -------------- 6.0/9.6 MB 3.7 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.6 MB 3.9 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.6 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 4.1 MB/s  0:00:02
   ------------------

In [49]:
from ragas.llms import LlamaIndexLLMWrapper

# you already defined `llm = HuggingFaceLLM(...)` and `Settings.llm = llm`

evaluator_llm = LlamaIndexLLMWrapper(llm)


C:\Users\AZhaa\AppData\Local\Temp\ipykernel_2132\843810481.py:5: DeprecationWarning: LlamaIndexLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluator_llm = LlamaIndexLLMWrapper(llm)


In [50]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(
    "sentence-transformers/LaBSE",
    device="cuda" if torch.cuda.is_available() else "cpu"
)


In [51]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/LaBSE",
    model_kwargs={"device": "cuda"},
)


C:\Users\AZhaa\AppData\Local\Temp\ipykernel_2132\3569151.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [52]:
# Starting from df_sample with: context, question, ground_truth, model_answer
df_ragas = df_done.copy()

# Rename columns to match Ragas expectations
df_ragas["retrieved_contexts"] = df_ragas["context"].apply(lambda x: [x])  # must be list of strings
df_ragas["response"] = df_ragas["model_answer"]
df_ragas["user_input"] = df_ragas["question"]
df_ragas["reference"] = df_ragas["ground_truth"]  # rename ground_truth to reference

print(df_ragas.columns)
# Should include: 'user_input', 'response', 'retrieved_contexts', 'reference'


Index(['context', 'question', 'ground_truth', 'model_answer',
       'retrieved_contexts', 'response', 'user_input', 'reference'],
      dtype='object')


In [53]:

from ragas import EvaluationDataset
ragas_dataset = EvaluationDataset.from_pandas(df_ragas)


In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness,
)

from ragas import evaluate

result = evaluate(
    ragas_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        answer_correctness,
    ],
    llm=evaluator_llm,
    embeddings=embeddings,
)
result


C:\Users\AZhaa\anaconda3\envs\nlp_gpu\lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)
n values greater than 1 not support for LlamaIndex LLMs0 [00:00<?, ?it/s]
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


### Overall RAGAS Scores

In [ ]:
import pandas as pd

overall_df = result.to_pandas()
overall_df.to_csv("ragas_overall_scores.csv", index=False)

print("Saved: ragas_overall_scores.csv")


### Per-Question RAGAS Scores

In [ ]:
from ragas import evaluate

per_question = evaluate(
    df_sample,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        answer_correctness,
    ],
    return_multi=True 
)


In [ ]:
per_question_df = per_question.to_pandas()
per_question_df.to_csv("ragas_per_question_scores.csv", index=False)

print("Saved: ragas_per_question_scores.csv")


In [ ]:
ragas_df = result.to_pandas()
ragas_df


## Bar Chart + Save

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(ragas_df["metric"], ragas_df["value"], color="#4C72B0")
plt.ylim(0, 1)
plt.title("RAGAS Metrics Overview", fontsize=16)
plt.ylabel("Score", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.grid(axis="y", linestyle="--", alpha=0.4)

plt.tight_layout()
plt.savefig("ragas_bar_chart.png", dpi=400)
plt.show()

print("✔ Saved: ragas_bar_chart.png")


## Line Chart + Save

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(ragas_df["metric"], ragas_df["value"], marker="o", linewidth=2, color="#55A868")
plt.ylim(0, 1)
plt.title("RAGAS Metrics Trend", fontsize=16)
plt.ylabel("Score", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.grid(True, linestyle="--", alpha=0.4)

plt.tight_layout()
plt.savefig("ragas_line_chart.png", dpi=400)
plt.show()

print("✔ Saved: ragas_line_chart.png")


## Radar Chart (Spider Plot) + Save

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

metrics = ragas_df["metric"].tolist()
values = ragas_df["value"].tolist()

# Close the circle
values += values[:1]
angles = np.linspace(0, 2*np.pi, len(values))

plt.figure(figsize=(8, 8))
ax = plt.subplot(111, polar=True)

ax.plot(angles, values, linewidth=2, color="#C44E52")
ax.fill(angles, values, alpha=0.25, color="#C44E52")

ax.set_thetagrids(angles[:-1] * 180/np.pi, metrics)
ax.set_title("RAGAS Radar Chart", fontsize=16)
ax.grid(True, linestyle="--", alpha=0.4)

plt.tight_layout()
plt.savefig("ragas_radar_chart.png", dpi=400)
plt.show()

print("✔ Saved: ragas_radar_chart.png")
